### LOFAR calibration solutions
#### The aim is to investigate the ionospheric signal extracted during the calibration routine


First we load the numpy file that contains data and time variables. 
The 'data' variable has ionospheric phases converted into dTEC for core and remote baselines

In [ ]:
import numpy as np

In [ ]:
file=np.load('L80898.npz',allow_pickle=True)

The above returns a dictionary-like object, containing {filename: array} key-value pairs, one for each file in the archive
We check the variables that exist in the file:

In [ ]:
for k in file.keys():
    print(k)

We can extract these variables separately by using:

In [ ]:
data=file['data']
times=file['times']

Or we can extract them with the following code; it creates variables with the same names as keys in the original file 

In [ ]:
for k, v in file.items():
    exec(f"{k} = v")

We can take a closer look at the dtec solutions

In [ ]:
print(data.shape)

This observation has 2876 time samples of ionospheric solutions for 56 baselines. Let's plot them to see how variable is the ionosphere during this observation.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig,axs=plt.subplots()
axs.plot(np.arange(len(data[0,:])),data.T)
axs.set_xlabel('sample number')
axs.set_ylabel('dTEC [TECU]')
plt.show()
plt.close()

We can see that different baselines detect similar patterns; since there are both core and remote stations included, let's plot them separately. To do this, we need to have information on the station names, which can be found in the 'stations.npz' file. 

##### Exercise 1:
Load the stations.npz file, check existing variables 

In [ ]:
file2=np.load('stations.npz',allow_pickle=True)

The stations' names are stored as the 'stats' variable, so let's check them

In [ ]:
stations=file2['stats']
print(stations)

We can remove the formatting 'b' letter with

In [ ]:
stations=np.array([stats.decode() for stats in stations])
print(stations)

The core stations start with 'CS', while remote stations with 'RS'. Let's set the reference station as CS001HBA0:

In [ ]:
data=data-data[0,:]

We can now plot these two types separately (change the index for data to include relevant baselines):

In [ ]:
fig,axs=plt.subplots(2)
axs[0].plot(np.arange(len(data[0,:])),data[1,:].T)
axs[1].plot(np.arange(len(data[0,:])),data[54:,:].T)
axs[1].set_xlabel('sample number')
axs[1].set_ylabel('dTEC [TECU]')
axs[0].set_ylabel('dTEC [TECU]')
plt.show()
plt.close()

Let's now take a look at spatial distribution of the dTEC.

We need to know the positions of stations, we will use the librabry 'lofarantpos.db'

In [ ]:
import lofarantpos.db

In [ ]:
db = lofarantpos.db.LofarAntennaDatabase()
coord=np.array([db.phase_centres[stat] for stat in stations])
print(stations[0],coord[0])

Having coordinates of each station, we can plot now the dTEC for each baseline at its endpoint for time sample number t_ind. 

Make plots for few values of t_ind, check if you can see some specific direction or the solutions look random. Play with normalization of the color scale. 

In [ ]:
import matplotlib.colors as cm

In [ ]:
t_ind=2500

fig,axs=plt.subplots()
cl=axs.scatter((coord[:,1]-coord[0,1])/1000,(coord[:,0]-coord[0,0])/1000,c=data[:,t_ind],edgecolors='black',s=50,cmap='jet',norm=cm.Normalize(vmin=-0.5,vmax=0.2))
axs.set_xlim(-20,20)
axs.set_ylim(-60,30)
axs.set_ylabel('N [km]')
axs.set_xlabel('E [km]')
axs.set_aspect('equal')
fig.colorbar(cl,label='dTEC [TECU]')
axs.set_title('Remote stations')
plt.show()
plt.close()

fig,axs=plt.subplots()
cl=axs.scatter((coord[:-11,1]-coord[0,1])/1000,(coord[:-11,0]-coord[0,0])/1000,c=data[:-11,t_ind],edgecolors='black',s=50,cmap='seismic',norm=cm.Normalize(vmin=-0.01,vmax=0.01))
axs.set_xlim(-2,2)
axs.set_ylim(-2,2)
axs.set_ylabel('N [km]')
axs.set_xlabel('E [km]')
axs.set_aspect('equal')
axs.set_title('Core stations')
fig.colorbar(cl,label='dTEC [TECU]')
plt.show()
plt.close()

Up till now we were using sample numbers, but we want to convert it into time. 

The variable 'times' contains the time for each sample expressed in MJD but using seconds instead of days.
We need to convert these time instances into days, and use astropy library to extract the date and time

In [ ]:
print(times[:10])

We can check the sampling time in second:

In [ ]:
time_l=np.diff(times)
print(time_l[:10])
step=int(time_l[0])
print(step)

We can now plot the observation in local time

In [ ]:
lt=np.arange(len(data[0,:]))*step/3600

In [ ]:
fig,axs=plt.subplots()
axs.plot(lt,data.T)
axs.set_xlabel('Time [h]')
axs.set_ylabel('dTEC [TECU]')
plt.show()
plt.close()

We will extract now the time in UTC

In [ ]:
from astropy.time import Time

In [ ]:
MJD=times/(3600.*24.)   
T=Time(MJD, format='mjd', scale='utc')   #creating astropy time object

To get UTC date and time, we use ISO formatting:

In [ ]:
Ti=T.iso
print(Ti[0])

Splitting the above into date and time:

In [ ]:
date=Ti[0].split(' ')[0].split('-')
time=Ti[0].split(' ')[1]

#### Exercise 2
Extract the UTC time for all samples 

In [ ]:
utc_t=np.array([Tii.split(' ')[1] for Tii in Ti])
print(utc_t[:10])

#### Exercise 3
Make a spatial plot of dTEC for a chosen t_ind, convert the t_ind to UTC and add it to the plot (can be added to the title)


In [ ]:
t_ind=2000
tu=utc_t[t_ind][:8]

fig,axs=plt.subplots()
cl=axs.scatter((coord[:-11,1]-coord[0,1])/1000,(coord[:-11,0]-coord[0,0])/1000,c=data[:-11,t_ind],edgecolors='black',s=50,cmap='seismic',norm=cm.Normalize(vmin=-0.01,vmax=0.01))
axs.set_xlim(-2,2)
axs.set_ylim(-2,2)
axs.set_ylabel('N [km]')
axs.set_xlabel('E [km]')
axs.set_aspect('equal')
axs.set_title('Core stations')
fig.colorbar(cl,label='dTEC [TECU]')
plt.title(str(tu))
plt.show()
plt.close()

### Rotation measure file

Let's now investigate different product of calibration - Rotation Measure RM

#### Exercise 4

Load the file and extract the variables

Make a plot of RM for all the stations using local time

In [ ]:
rmfile=np.load('rmtime.npz')
for k in rmfile.keys():
    print(k)

In [ ]:
for k, v in rmfile.items():
    exec(f"{k} = v")

In [ ]:
fig,axs=plt.subplots()
axs.plot(lt,rmtime[:,47:])
axs.set_xlabel('Time [h]')
axs.set_ylabel('RM [rad]')
plt.show()
plt.close()

We can check which station detected the largest RM amplitude, and at which time sample: 

In [ ]:
max_ind=np.unravel_index(np.argmax(abs(data)),data.shape)
print(max_ind)